#Estevan R. Tomazini - PUCPR
#Depertamento de TI
#Curso de BigData e inteligência analítica
#Atividade feita individualmente
 



## O que foi feito antes:

##
## Na ATP 1 tentei modelo de classificação porém o resultado verificado
## foi bem a baixo do esperado, acredito que modelo de regressão
## parece ser mais adequado para prever a taxa de vitória "WinShare".
#

## Novamente serão importadas as bibliotecas principais no primeiro passo.




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas_profiling import ProfileReport # faz perfil de database
import sklearn
from sklearn.model_selection import train_test_split # utiliza split entre os testes
from sklearn.preprocessing import RobustScaler # todas as entradas estarão na mesma escala numerica
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
## leitura do DF a partir do método read.csv() da biblioteca pandas
## como foi feito no google colab, por razões do voilà ter acabado com
## meu Jupyter lab, que simplismente parou de funcionar.

## ! professor: lembre de mostrar o acesso à VM. Fez falta e não havia informações
## de como acessá-la. 

## o arquivo foi guardado na memória de trabalho da maquina

In [2]:
# 1 - Coleta dos dados:

df = pd.read_csv('/content/nba_stats.csv')

In [ ]:
## A seguir será mostrada a estrutura do DF sendo analizado, por estar no
## google colab, a interface própria (voìla) permite a visualização sem a
## necessidade de funções/métodos adicionais, print(), head(), tail(), info() e afins
## apenas utilizando a 'magic point'.

#### intenção do modelo é prever a métrica 'WinShares'.

In [3]:
df


,Ano,Posicao,Idade,Jogos,TrueShootingPercentage,FreeThrowRate,OffensiveWinShares,DefensiveWinShares,WinShares,FieldGoals,...,TwoPointFieldGoals,TwoPointFieldGoalAttempts,TwoPointFieldGoalPercentage,EffectiveFieldGoalPercentage,FreeThrows,FreeThrowAttempts,FreeThrowPercentage,Assists,PersonalFouls,Points
0,1950,G-F,31.0,63,0.368,0.467,-0.1,3.6,3.5,144,...,144,516,0.279,0.279,170,241,0.705,176,217,458
1,1950,SG,29.0,49,0.435,0.387,1.6,0.6,2.2,102,...,102,274,0.372,0.372,75,106,0.708,109,99,279
2,1950,SF,25.0,67,0.394,0.259,0.9,2.8,3.6,174,...,174,499,0.349,0.349,90,129,0.698,140,192,438
3,1950,F,24.0,15,0.312,0.395,-0.5,-0.1,-0.6,22,...,22,86,0.256,0.256,19,34,0.559,20,29,63
4,1950,F,24.0,13,0.308,0.378,-0.5,-0.1,-0.6,21,...,21,82,0.256,0.256,17,31,0.548,20,27,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24619,2017,PF,24.0,62,0.604,0.442,3.4,2.2,5.6,253,...,253,442,0.572,0.571,133,196,0.679,99,189,639
24620,2017,C,27.0,51,0.508,0.247,0.5,0.6,1.0,78,...,78,157,0.497,0.494,22,39,0.564,42,61,178
24621,2017,C,20.0,19,0.346,0.161,-0.1,0.1,0.0,10,...,10,31,0.323,0.323,3,5,0.600,4,17,23
24622,2017,SF,22.0,44,0.503,0.181,-0.3,0.8,0.5,88,...,55,122,0.451,0.473,31,40,0.775,36,78,240


In [ ]:
## Agora vamos retirar algumas colunas que não precisamos

In [4]:
df.head



<bound method NDFrame.head of         Ano Posicao  Idade  Jogos  TrueShootingPercentage  FreeThrowRate  \
0      1950     G-F   31.0     63                   0.368          0.467   
1      1950      SG   29.0     49                   0.435          0.387   
2      1950      SF   25.0     67                   0.394          0.259   
3      1950       F   24.0     15                   0.312          0.395   
4      1950       F   24.0     13                   0.308          0.378   
...     ...     ...    ...    ...                     ...            ...   
24619  2017      PF   24.0     62                   0.604          0.442   
24620  2017       C   27.0     51                   0.508          0.247   
24621  2017       C   20.0     19                   0.346          0.161   
24622  2017      SF   22.0     44                   0.503          0.181   
24623  2017       C   19.0     38                   0.547          0.206   

       OffensiveWinShares  DefensiveWinShares  WinShares 

In [5]:
## Mostrado o DataFrame e suas caracteristicas, 
## novamente utilizei o método reset_index() para evitar bugs
## que comumente acontecem.


df = df.reset_index()

In [ ]:
## 2 : Preparação dos dados:

In [6]:
df1 = df.dropna()

In [ ]:
## Agora não teremos mais problemas de variaveis infinitas e NaN,
## a partir da visualização é possível retirar colunas que representam 
## variáveis que dificultarão a análise e/ou modelo. 
## como são muitas colunas, escolhi retirar os indices inicialmente. 
## temos as colunas:

#[Ano]
#[Posicao]
#[Idade]
#[Jogos]
#[TrueShootingPercentage]
#[FreeThrowRate]
#[OffensiveWinShares]
#[DefensiveWinShares]
#[WinShares]
#[FieldGoals]
#[FieldGoalAttempts]
#[FieldGoalPercentage]
#[TwoPointFieldGoals]
#[TwoPointFieldGoalAttempts]
#[TwoPointFieldGoalPercentage]
#[EffectiveFieldGoalPercentage]
#[FreeThrows]
#[FreeThrowAttempts]
#[FreeThrowPercentage]
#[Assists]
#[PersonalFouls]
#[Points]

## Existe um calculo manual para WinShares, mostrado no site sport reference 
## (https://www.basketball-reference.com/about/ws.html). A partir dele,
## escolhi manter algumas das variáveis que descartei na ATP 1



In [7]:
df_tratado = df1.drop(['Ano', 'Posicao', 'FieldGoalAttempts', 'TwoPointFieldGoalAttempts', 'FreeThrowAttempts'], axis=1)

In [ ]:
## a exclusão de colunas visa tratar e diminuir a complexidade dos dados para o 
## desenvolvimento do modelo. Julgo que principalmente a posição e época jogada
## são dados pouco explicativos. Época jogada, embora possa ter correlação com
## a eficiência devido às regras, pode apenas dificultar a classificação
## dos jogadores
## Posição pode ser relevante mas por classificação deve dificultar
## mais do que explicar.

In [8]:
df_tratado

,index,Idade,Jogos,TrueShootingPercentage,FreeThrowRate,OffensiveWinShares,DefensiveWinShares,WinShares,FieldGoals,FieldGoalPercentage,TwoPointFieldGoals,TwoPointFieldGoalPercentage,EffectiveFieldGoalPercentage,FreeThrows,FreeThrowPercentage,Assists,PersonalFouls,Points
0,0,31.0,63,0.368,0.467,-0.1,3.6,3.5,144,0.279,144,0.279,0.279,170,0.705,176,217,458
1,1,29.0,49,0.435,0.387,1.6,0.6,2.2,102,0.372,102,0.372,0.372,75,0.708,109,99,279
2,2,25.0,67,0.394,0.259,0.9,2.8,3.6,174,0.349,174,0.349,0.349,90,0.698,140,192,438
3,3,24.0,15,0.312,0.395,-0.5,-0.1,-0.6,22,0.256,22,0.256,0.256,19,0.559,20,29,63
4,4,24.0,13,0.308,0.378,-0.5,-0.1,-0.6,21,0.256,21,0.256,0.256,17,0.548,20,27,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24619,24619,24.0,62,0.604,0.442,3.4,2.2,5.6,253,0.571,253,0.572,0.571,133,0.679,99,189,639
24620,24620,27.0,51,0.508,0.247,0.5,0.6,1.0,78,0.494,78,0.497,0.494,22,0.564,42,61,178
24621,24621,20.0,19,0.346,0.161,-0.1,0.1,0.0,10,0.323,10,0.323,0.323,3,0.600,4,17,23
24622,24622,22.0,44,0.503,0.181,-0.3,0.8,0.5,88,0.398,55,0.451,0.473,31,0.775,36,78,240


In [ ]:
## A complexidade já diminuiu de 22 colunas para 17, próximo comando mostra o tipo dos dados

In [9]:
df_tratado.dtypes

index                             int64
Idade                           float64
Jogos                             int64
TrueShootingPercentage          float64
FreeThrowRate                   float64
OffensiveWinShares              float64
DefensiveWinShares              float64
WinShares                       float64
FieldGoals                        int64
FieldGoalPercentage             float64
TwoPointFieldGoals                int64
TwoPointFieldGoalPercentage     float64
EffectiveFieldGoalPercentage    float64
FreeThrows                        int64
FreeThrowPercentage             float64
Assists                           int64
PersonalFouls                     int64
Points                            int64
dtype: object

In [ ]:
## Veremos os 20 maiores WinShares para ter noção
## da importância de algumas entidades
## é um pouco estranho trabalhar sem os nomes do jogadores
## como referência.

In [10]:
df_tratado.sort_values('WinShares',ascending=False).head(20)

,index,Idade,Jogos,TrueShootingPercentage,FreeThrowRate,OffensiveWinShares,DefensiveWinShares,WinShares,FieldGoals,FieldGoalPercentage,TwoPointFieldGoals,TwoPointFieldGoalPercentage,EffectiveFieldGoalPercentage,FreeThrows,FreeThrowPercentage,Assists,PersonalFouls,Points
3294,3294,24.0,81,0.603,0.363,18.3,7.1,25.4,1159,0.574,1159,0.574,0.574,504,0.689,370,235,2822
1948,1948,27.0,80,0.537,0.442,14.4,10.6,25.0,1204,0.524,1204,0.524,0.524,540,0.531,403,182,2948
427,427,26.0,68,0.509,0.453,16.0,7.4,23.4,678,0.428,678,0.428,0.428,576,0.803,208,308,1932
1694,1694,25.0,80,0.536,0.431,17.1,6.0,23.1,1597,0.506,1597,0.506,0.506,835,0.613,192,123,4029
3049,3049,23.0,82,0.606,0.370,17.0,5.3,22.3,1063,0.577,1063,0.577,0.577,470,0.690,272,264,2596
2349,2349,30.0,81,0.637,0.761,14.8,7.0,21.9,785,0.683,785,0.683,0.683,386,0.441,630,143,1956
3559,3559,25.0,76,0.580,0.260,14.4,7.5,21.9,982,0.554,982,0.554,0.554,328,0.713,379,208,2292
2223,2223,29.0,79,0.547,0.490,12.9,8.5,21.4,1074,0.540,1074,0.540,0.540,501,0.513,414,171,2649
8831,8831,24.0,82,0.603,0.430,15.2,6.1,21.2,1069,0.535,1062,0.546,0.537,723,0.841,485,270,2868
172,172,25.0,68,0.487,0.456,15.3,5.8,21.1,649,0.407,649,0.407,0.407,567,0.779,197,297,1865


In [ ]:
## Podemos observar que algumas variáveis parecem ser menos
## importantes na explicação da WinShare como 
## Assists e PersonalFouls, porém serão mantidas para verificar a aderência

In [11]:
## Não entendo porque rodou sem erros mas não mostrou o grafico
## manterei só para mostrar a tentativa

df_WS= df_tratado.sort_values('WinShares',ascending=False).head(20)

plt.figure(figsize=(10,6))
plt.barh('index', 'WinShares',data=df_WS)
plt.xlabel("index", size=15)
plt.ylabel('WinShares', size=15)
plt.title("Top 20 Winshares", size=18)
plt.gca().invert_yaxis()


plt.show()


In [12]:
## 3 : Seleção do modelo
## O modelo escolhido foi o método de aprendizado supervisionado de
## REGRESSÃO LINEAR

## Serão mostrados os dados que sobraram do tratamento inicial para então 
## iniciar a fase de testes:
## A variável que se deseja prever é WinShares, probabilidade de vitória,
## de um jogador em dados de ofensiva, defensiva e tempo em quadra 
## (NBAstuffer.com).
## Exemplo: 48/WS seria uma WinShare com 48 minutos jogados e representa
## um terço do TeamWinShares, que é distribuído entre os jogadores.

## Na ATP1, pela caracteristica dos dados, imaginei que o método de 
## classificação seria mais adequado, porém obtive apenas 25% no modelo.

## O teste e treino será dividido como anteriormente, 25% para a amostra.
df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23697 entries, 0 to 24623
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         23697 non-null  int64  
 1   Idade                         23697 non-null  float64
 2   Jogos                         23697 non-null  int64  
 3   TrueShootingPercentage        23697 non-null  float64
 4   FreeThrowRate                 23697 non-null  float64
 5   OffensiveWinShares            23697 non-null  float64
 6   DefensiveWinShares            23697 non-null  float64
 7   WinShares                     23697 non-null  float64
 8   FieldGoals                    23697 non-null  int64  
 9   FieldGoalPercentage           23697 non-null  float64
 10  TwoPointFieldGoals            23697 non-null  int64  
 11  TwoPointFieldGoalPercentage   23697 non-null  float64
 12  EffectiveFieldGoalPercentage  23697 non-null  float64
 13  F

In [13]:
## Train-Test-split: 

##Base de testes avalia desempenho do algoritmo. 
## conta com reproducibilidade random_state = 100% (totalmente randomico)


## Prever (y) WinShares baseado em (x): Idade, Jogos, TrueShootingPercentage, 
## FreeThrowRate, OffensiveWinShares, DefensiveWinShares, FieldGoals, 
## FieldGoalPercentage, TwoPointFieldGoals, TwoPointFieldGoalPercentage, 
## EffectiveFieldGoalPercentage, FreeThrows, FreeThrowPercentage, Assists,
## PersonalFouls, Points.


## excluindo alguns indices e tentarívas.



y = df_tratado['WinShares']
X = df_tratado[['Idade', 'Jogos', 'TrueShootingPercentage', 'FreeThrowRate', 'OffensiveWinShares', 'DefensiveWinShares', 'FieldGoals', 'FieldGoalPercentage','TwoPointFieldGoals','TwoPointFieldGoalPercentage','EffectiveFieldGoalPercentage', 'FreeThrows','FreeThrowPercentage','Assists','PersonalFouls', 'Points']]


from sklearn.model_selection import train_test_split

num_test = 0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=num_test, random_state=100)

In [14]:
## MinMaxScaler
## Esse estimador escala e traduz cada variável individualmente
## de modo que esteja adequada a divisão de treino.
#

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

scale = MinMaxScaler()
model = LinearRegression()

X_train_sc = scale.fit_transform(X_train)
X_test_sc = scale.fit_transform(X_test)

In [15]:
## Ajuste de modelo

model.fit(X_train_sc, y_train)

model.coef_

array([ 6.85493257e-03,  9.55152775e-04,  1.51084433e-02,  3.41768254e-02,
        2.33943004e+01,  1.69649992e+01,  2.35810042e-02, -1.06098450e-02,
       -1.62810938e-02, -2.65280122e-03, -2.34408916e-03, -2.27251852e-02,
       -4.18932031e-03, -2.53766583e-03,  2.24399502e-05,  2.97563521e-02])

In [16]:
## preliminar

predict = model.predict(X_test_sc)

# A acurácia já se mostrou melhor do que o experimento treinado da ATP1
## com em torno de 40% de acurácia

print((mean_squared_error(y_test, predict)))

0.6538192798252555


In [17]:
## Predição de valores de WinShares
predict[0:10]

array([ 1.04950626,  6.27897953,  2.14247272,  2.90671033, -0.92948773,
        0.60336224, -1.27264674,  0.1052651 , -0.22313189, 13.69260238])

In [18]:
## Compara com mostra alguns dos indices com valores de WinShares
y_test[0:10]

22255     1.8
20490     6.3
345       2.8
4451      3.4
23226     0.1
1330      1.4
5898     -0.2
8248      1.0
29        0.7
6950     12.5
Name: WinShares, dtype: float64

In [ ]:
## 4 : Métricas

## testei algumas métricas para desambiguar depois de verificar 
## valor de R quadrado próximo de 1, mostrando boa acurácia.



In [19]:
## Metrica R quadrado:
## Gerou resultados ótimos, próximos a 1,0.
## indicando valores próximos 


from sklearn.metrics import explained_variance_score, r2_score

# Perfil de predição com score muito melhor
print(explained_variance_score(y_test, predict, multioutput='raw_values'))



[0.97132093]


In [20]:
print(r2_score(y_test, predict))

## boa estimativa, próxima de 1

0.9323424142090423


In [21]:
display(predict)

## Mostrando array dos WinShares preditos

array([ 1.04950626,  6.27897953,  2.14247272, ..., -0.34628212,
        7.02440852,  7.18077568])

In [44]:
## Pipeline:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV # utilizado para o split entre treinamento e teste e para a otimização de hiperparâmetros
from sklearn.ensemble import * # importando vários ensembles para que possamos testá-los posteriormente
from sklearn.pipeline import Pipeline # utilizado para criar pipelines
from sklearn.neighbors import KNeighborsClassifier # utilizado para treinar o KNN
from sklearn.linear_model import LogisticRegression # utilizado para treinar um modelo de classificação (regressão logística - apesar do nome é para problemas de classificação)
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler # utilizado para converter colunas do tipo texto para numéricas
from sklearn.decomposition import PCA # utilizado para fazer o PCA nos pipelines
from sklearn.metrics import f1_score, make_scorer # utilizado para calcular a performance dos pipelines
from lightgbm import LGBMClassifier # utilizado para treinar o LightGBM
from sklearn import set_config # utilizado para mostrar os passos do pipeline de forma visual

## Testei vários códigos de pipelines, inclusive aqueles
## do script da aula, nenhum deles rodou e não consigo compreender a razão
## Mesmo problema com os gráficos.

## infelizmente o tempo está acabando e não poderei enviar com o teste 
## feito corretamente utilizando pipeline como outras formas de modelos.



In [ ]:
pipe = Pipeline([('Encoder', MinMaxScaler()), 'Modelo', LinearRegression()]).fit(X_train, y_train)

## Esse foi o primeiro teste

In [ ]:
## testando metricas

In [ ]:
# Aparentemente escolhi bem as variáveis para o modelo de regressão. 

# Utilização das porcentagens facilitou. 

In [ ]:
## Parte 5: Ajuste
## Métricas

## Para testar a acurácia do método, ou ainda, se o resultado é representativo
## dos dados ou é apenas produto do acaso.

## Como o modelo desenvolvido foi de Regressão linear,
## as métricas utilizadas serão o R quadrado, que é o metrica mais simples de 
## comprender, já possível de visualizar no Excel e o Erro médio absoluto %
## ou MAPE.

## Erro -> diferença entre o valor gerado e o real.

## R quadrado é um valor entre 0 e 1 (quanto mais próximo de 1, melhor), 
## funciona em modelos que não utilizam varios datasets e 
## contam com varias variáveis porém precisa ser associado a outras metricas.

## MAPE é utilizado onde valores muito discrepantes devem ser penalizados
## quanto mais próximo de 0, melhor e valores próximos de 0 tendem a 
## aumentar muito o MAPE
 

In [53]:
## MAPE
## Existem valores proximos de zero, inflam o resultado do MAPE.
## nesse caso, talvez devesse ter retirado os valores próximos de zero
## mas isso me soa como manipulação de dados, visto que alguns zeros
## podem ser muito relevantes

from sklearn.metrics import mean_absolute_percentage_error

print(mean_absolute_percentage_error(y_test, predict))

289197173100280.75


In [54]:
## MAE - mean absolute error - erro absoluto médio:

## quanto mais próximo de 0, melhor. Mais sucetivel a outliers do que o MSE.
## o resultado de 0,73 é normalmente muito bom, mostrando certa confiabilidade, 
## pricnipalmente a outras métricas, como o R quadrado, que embora não seja
## uma métrica muito apreciada, é razoavelmente compreensível para iniciantes.


from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_test, predict))


0.7351694156329871


In [55]:
## MSE - mean squared error - erro médio quadrático:

## verifica a média dos erros quadráticos, com valores próximos de 0
## representando modelo mais acurado. É mais sensível a outliers do que MAE.
##



from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, predict))

0.6538192798252555


In [ ]:
## R quadrado alta e outras métricas, com excesão da MAPE, baixas mostram que
## as variáveis escolhidas explicam bem "WinShares", de modo que o modelo 
## é satisfatório com essa variáveis

In [ ]:
## Essa foi minha disciplina favorita até agora, tenho vontade de seguir 
## essa area. Infelizmente a estrutura modular da PUC tende a nos forçar 
## a focar em algumas disciplinas ou em resolver problemas com servidores
## de VM de algumas disciplinas. Quando vê, as disciplinas com mais material
## acabam negligênciadas.

In [ ]:
## ultima célula: conversão para HTML

##
## HTML

%%shell
jupyter nbconvert --to html /content/ATP1_TOMAZINI_ER_ModeloBasketClassificacao.ipynb